In [1]:
%pwd
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: GeForce GTX 760 (CNMeM is disabled, cuDNN 5105)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
import os
base= os.getcwd() + "/data/redux"
# base= os.getcwd() + "/data/redux/sample"
train= base+"/train"
test= base+"/test"
valid= base+"/valid"
results_path = base+"/results"

In [3]:
from matplotlib import pyplot as plt
import bcolz
def plot(img):
    plt.imshow(img)
def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

In [4]:

    
import json   
import keras.preprocessing.image as preprocess
from keras.models import Sequential
import numpy as np
from keras.layers.core import Lambda, Flatten, Dropout, Dense
from keras.layers.convolutional import ZeroPadding2D, Convolution2D, MaxPooling2D
from keras.utils.data_utils import get_file
from keras.optimizers import Adam


vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.

        Args: 
            x: Image array (height x width x channels)
        Returns:
            Image array (height x width x transposed_channels)
    """
    x = x - vgg_mean
#     print x
#     import pdb;pdb.set_trace()
    return x[:, ::-1] # reverse axis rgb->bgr

class hgg:
    def get_classes(self):
        fname = 'imagenet_class_index.json'
        fpath = get_file(fname, self.FILE_PATH+fname, cache_subdir='models')
        with open(fpath) as f:
            class_dict = json.load(f)
        self.classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

    def get_batches(self,path, shuffle=True, batch_size=8, rotation_range=10 ,width_shift_range=10, height_shift_range=10, shear_range=0.3 ):
#         imageGenerator doc at https://keras.io/preprocessing/image/
        imageGenerator = preprocess.ImageDataGenerator(
#             rotation_range=rotation_range,
#             width_shift_range=width_shift_range,
#             height_shift_range=height_shift_range,
#             shear_range=shear_range,
#             zoom_range=0
        )
        return imageGenerator.flow_from_directory(
            shuffle=shuffle,
            directory=path, 
            target_size=(224,224),
            batch_size=batch_size,
            class_mode='categorical'
        )
    def __init__(self):
        self.FILE_PATH = 'http://files.fast.ai/models/'

        self.createModel()
        self.get_classes()
        pass
    def createModel(self,):
        # Sequential doc at https://keras.io/getting-started/sequential-model-guide/
        self.model=Sequential()
        self.model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))
        self.addConvLayer(2,64)
        self.addConvLayer(2,128)
        self.addConvLayer(3,256)
        self.addConvLayer(3,512)
        self.addConvLayer(3,512)
        
        self.model.add(Flatten())
        
        self.model.add(Dense(4096,activation='relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(4096,activation='relu'))
        self.model.add(Dropout(0.5))
        
        
        self.model.add(Dense(1000,activation='softmax'))
        
        
        fname="vgg16.h5"
        self.model.load_weights(get_file(fname, self.FILE_PATH+fname, cache_subdir='models'))
        
    def addConvLayer(self,layers,filters):
        # Convolution2D https://keras.io/layers/convolutional/
        for i in range(layers):
            self.model.add(ZeroPadding2D((1,1)))
            self.model.add(Convolution2D(filters,3,3,activation='relu'))
        self.model.add(MaxPooling2D((2,2),strides=(2,2)))
        
    def ModifyLastLayerToNewOutput(self, no_of_outputs):
        self.model.pop()
        for layer in self.model.layers: layer.trainable=False
        self.model.add(Dense(no_of_outputs,activation='softmax'))
        print no_of_outputs
        self.compile()
    def finetune(self,batches):
        self.ModifyLastLayerToNewOutput(batches.nb_class)
        classes = list(iter(batches.class_indices)) # get a list of all the class labels
        
        # batches.class_indices is a dict with the class name as key and an index as value
        # eg. {'cats': 0, 'dogs': 1}

        # sort the class labels by index according to batches.class_indices and update model.classes
        for c in batches.class_indices:
            classes[batches.class_indices[c]] = c
        self.classes = classes
        
        
    def compile(self, lr=0.001):
        self.model.compile(optimizer=Adam(lr=lr), loss='categorical_crossentropy',metrics=['accuracy'])
    def fit(self,train_batches, validation_batches):
        self.model.fit_generator(train_batches, 
                        samples_per_epoch = train_batches.nb_sample,
                        nb_epoch=1, 
                        validation_data = validation_batches,
                        nb_val_samples = validation_batches.nb_sample,
                       )
   
h = hgg()
# from vgg16 import Vgg16

# h=Vgg16()

Using Theano backend.


In [5]:
train_batches = h.get_batches(train)
validation_batches = h.get_batches(valid)
test_batches = h.get_batches(test)
# validation_batches
# plot(validation_batches)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [6]:
h.finetune(train_batches)



2


In [ ]:
train_now=True
no_of_epochs=4
if train_now==True:
    latest_weights_filename = None
    for epoch in range(no_of_epochs):
        print "Running epoch: %d" % epoch
        if(epoch>=3):
            h.model.optimizer.lr = 0.001
        h.fit (train_batches, validation_batches)
        latest_weights_filename = 'ft%d.h5' % epoch
        h.model.save_weights(results_path+latest_weights_filename)
    print "Completed %s fit operations" % no_of_epochs
else:
    print "loading from saved_model"
    h.model.load_weights(results_path+"ft4.h5")


Running epoch: 0
Epoch 1/1
23000/23000 [==============================] - 1156s - loss: 0.1899 - acc: 0.9716 - val_loss: 0.2096 - val_acc: 0.9765

In [ ]:
preds=h.model.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
is_dog = np.round(1- preds)
print is_dog[:5]
filenames = test_batches.filenames
print filenames[:5]

In [ ]:
with open("submit.txt","a") as f:
    f.write("id,label\n")
    for i in range(len(preds)):
        if(is_dog[i][0]==1):
            is_dog[i][0]=0.95
        else:
            is_dog[i][0]=0.05
        print filenames[i][8:-4],",",is_dog[i][0]
        f.write("%s,%s\n"%(str(filenames[i][8:-4]),str(is_dog[i][0])) )

In [ ]:
from PIL import Image
idx=1
print preds[idx]
Image.open(test+"/" + filenames[idx])

In [ ]:
save_array(results_path + '/test_preds.dat', preds)
save_array(results_path + '/filenames.dat', filenames)